# Blondy's Soaring Prediction

Use herbie to build out a windgram, copying all the functionality of RASP by Andrew Berkley and Peter Spear

In [1]:
from herbie import Herbie
import xarray as xr
import numpy as np
import os, shutil

import matplotlib.pyplot as plt
from herbie.toolbox import EasyMap, pc
import cartopy.crs as ccrs
import cartopy.feature as feature
import pandas as pd
import matplotlib as mpl

recent = pd.Timestamp("now").floor("6h") #- pd.Timedelta("6H")
fxx = range(1,10)
data_dir = "/Users/andrew/data/hrdps/"

 ╭─▌▌Herbie─────────────────────────────────────────────╮
 │ WARNING: Unable to create config file               │
 │      /Users/andrew/.config/herbie/config.toml        │
 │ Herbie will use standard default settings.           │
 │ Consider setting env variable HERBIE_CONFIG_PATH.    │
 ╰──────────────────────────────────────────────────────╯



In [2]:
# select locations to plot (lumby, revelstoke) by specifying their location in decimal degrees (E, N)
locations = {"coopers":{"longitude":-118.986010, "latitude":50.285681},
             "revelstoke":{"longitude":-118.092554,"latitude":50.959530},
            }


In [3]:
# pick out the variables we need for rasp and their respective levels
levels = ["ISBL_0550",           
          "ISBL_0600",           
          "ISBL_0650",
          "ISBL_0700",           
          "ISBL_0750",
          "ISBL_0800",
          "ISBL_0850",
          "ISBL_0875",
          "ISBL_0900",
          "ISBL_0925",
          "ISBL_0950",
          "ISBL_0970",
          "ISBL_0985",
          "ISBL_1015",
         ]

variables = {"TCDC":"Sfc",
             "HPBL":"Sfc",
             "PRATE":"Sfc",
             "PRES":"Sfc",
             "ORGPHY":"Sfc",
            }
             #"HGT":levels,
             #"HGT":levels,
             #"TMP":levels,
             #"WIND":levels,
             #"WDIR":levels,
             #}

In [4]:
def isolate_latlon(ds, latitude=None, longitude=None, locations=None):
    """
    takes in a dataset and pares it down to a single column at lat, lon, (decimal deg)
    or takes in a dictionary of {"location":[lat, lon]} and returns dictionary {"location":dataset}
    """
    if locations == None:
        # select the nearest location to (lat, lon) (optimized for HRDPS)
        ds = ds.sel((((ds.longitude - longitude) ** 2 
             + (ds.latitude - latitude) ** 2) ** 0.5).argmin(dim=["x", "y"]))
    
    elif (lat, lon) == (None, None): # not debugged yet
        # select the nearest location to (lat, lon) (optimized for HRDPS)
        locations = {loc: ds.sel((((ds.longitude - locations[loc]["longitude"]) ** 2 + (ds.latitude - locations[loc]["latitude"]) ** 2) ** 0.5).argmin(dim=["x", "y"])) for loc in locations}
    
    else:
        raise ValueError("invalid lat=lat, lon=lon or locations={'my_location':{'latitude':xx, 'longitude':xx}")
        
    return ds

In [5]:
locations = {"revelstoke":{"longitude":-118.986010, 
                           "latitude":50.285681}}
#isolate_latlon(ds, longitude=-118.986010, latitude=50.285681)
#isolate_latlon(ds, locations)

In [6]:
def get_variable(time, model, product, fxx, variable, level):
    """
    wrapper for herbie object, outputs an xarray
    """
    
    # get the desired layer
    ds = Herbie(time,
                model=model,
                fxx=fxx,
                product=product,
                variable=variable,
                level=level,
                ).xarray()

    # if the variable name is unknown, assign it
    if "unknown" in ds:
        ds = ds.rename({"unknown":variable})
    
    # pare down 
    ds = isolate_latlon(ds, longitude=-118.986010, latitude=50.285681)
                
    return ds

In [9]:
def get_level(time, model, product, fxx, variable, level):
    """
    wrapper for get_variable, merges variables along isobaric levels
    """
    # clear the data folder
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    
    if len(level) > 1 and "ISBL" in level[0]: 
        ds = xr.concat([get_variable(time, 
                                     model=model, 
                                     product=product, 
                                     fxx=fxx, 
                                     variable=variable, 
                                     level=lev) for lev in level], dim="isobaricInhPa")
    else:
        ds = get_variable(time, 
                          model=model, 
                          product=product, 
                          fxx=fxx, 
                          variable=variable, 
                          level=level)
        
    return ds

In [10]:
print(pd.Timestamp("now"))
ds = xr.concat([xr.merge([get_level(recent, model="hrdps", fxx=f, product="continental/2.5km", variable=v, level=variables[v]) for v in variables]) for f in fxx], dim="valid_time")
print(pd.Timestamp("now"))


2024-11-27 19:11:37.695667
✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Nov-27 18:00 UTC F01 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241127]


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Nov-27 18:00 UTC F01 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241127]


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Nov-27 18:00 UTC F01 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241127]


...

✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Nov-27 18:00 UTC F09 ┊ GRIB2 @ msc ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241127]


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


✅ Found ┊ model=hrdps ┊ product=continental/2.5km ┊ 2024-Nov-27 18:00 UTC F09 ┊ GRIB2 @ msc2 ┊ IDX @ None
👨🏻‍🏭 Created directory: [/Users/andrew/data/hrdps/20241127]


/opt/miniconda3/envs/herbie/lib/python3.13/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: '/Users/andrew/data/hrdps/20241127/20241127T18Z_MSC_HRDPS_TCDC_Sfc_RLatLon0.0225_PT001H.grib2'

In [ ]:
tstore = [] # timesteps
for fxx in np.arange(1,8): 
    vstore = [] # variable names
    for var in variables:
        print(var)
        lstore = [] # variable level (z)
        for level in variables[var]:
            __ds = Herbie(
                recent,
                model="hrdps",
                fxx=int(fxx),
                product="continental/2.5km",
                variable=var,
                level=level,
                ).xarray()

            # if the variable name is unknown, assign it
            if "unknown" in __ds:
                __ds = __ds.rename({"unknown":var})

            lstore.append(__ds)
        if len(variables[var]) > 1: # this only works for pressure levels at present. caution.
            __ds = xr.concat(lstore, dim="isobaricInhPa")
        vstore.append(__ds)  
    _ds = xr.merge(vstore)
    tstore.append(_ds)
ds = xr.concat(tstore, dim="valid_time") # merge timesteps

In [ ]:
ds["lr"] = ds.t.diff(dim="isobaricInhPa") / ds.gh.diff(dim="isobaricInhPa") # calculate lapse rate (degC/m)

In [ ]:
location="revelstoke"
# select the closest pixel to "revelstoke"
# HRDPS doesnt grid on lat/lon, so this becomes necessary
revelstoke = ds.sel((((ds.longitude - locations[location]["longitude"]) ** 2 
       + (ds.latitude - locations[location]["latitude"]) ** 2) ** 0.5).argmin(dim=["x", "y"]))
revelstoke = revelstoke.roll(valid_time=-8) #tz_localize('UTC').tz_convert('America/Vancouver') # make this less naive

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
lr_cmap = mpl.colors.ListedColormap(['#ff0000',
                                     '#ffa500',
                                     '#ff69b4',
                                     '#9370db',
                                     '#eee8aa',
                                     '#b0c4de',
                                     '#DCDCDC',
                                     '#808080',
                                    ])
lr_bounds = [-0.0098, -0.0085, -0.00725, -0.006, 0.000, 0.0025, 0.005]
lr_norm = mpl.colors.BoundaryNorm(lr_bounds, lr_cmap.N)
ax.contourf(revelstoke.valid_time, revelstoke.gh[0].values, revelstoke.lr.T, cmap=lr_cmap)
#ax.invert_yaxis()
#ax.set_yticks(ticks=revelstoke.isobaricInhPa.values, labels=revelstoke.gh[0].values)
